# Implementing LSTM Approach

In [16]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)

In [ ]:
X = torch.tensor(dataX, dtype=torch.float)

In [ ]:
# normalize
X = X / float(len(alphabet))

In [ ]:
X.shape

In [ ]:
y = torch.tensor(dataY, dtype=torch.long)

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
N = X.shape[0]

In [ ]:
# RNN - LSTM
class Lstm(nn.Module):
    def __init__(self,  embeded_dim, hidden_dim, d_out):
        super(Lstm, self).__init__()
        self.embeded_dim = embeded_dim
        self.hidden_dim = hidden_dim
        self.d_out = d_out

        self.lstm = nn.LSTM(self.embeded_dim, self.hidden_dim)
        self.out = nn.Linear(self.hidden_dim, self.d_out)

    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))
    
    def forward(self, cell_input):
        lstm_out, self.hidden = self.lstm(cell_input, self.hidden)
        score = self.out(lstm_out)
        return score

In [ ]:
learning_rate = 0.005

n_epochs = 1000
batch_size = 6

# Regularisierung
weight_decay=0.0

# the model
hidden_dim = 16
embeded_dim = 1
model = Lstm(embeded_dim, hidden_dim, d_out=len(alphabet))

In [ ]:
# ADAM
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
# Loss
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train
loss_hist = []
epochs = range(n_epochs)
idx = 0

for t in epochs:
    for batch in range(0, int(N/batch_size)):
        # Step 1. Calculate Batch
        batch_x = X[batch * batch_size : (batch + 1) * batch_size, :]
        
        # convert to: sequence x batch_size x n_features 
        batch_x = batch_x.reshape(batch_size, seq_length, 1).transpose(0,1)
        
        batch_y = y[batch * batch_size : (batch + 1) * batch_size]                
        
        # Step 2. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
            
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden(batch_size)
        
        # Step 3. Run our forward pass.
        output = model(batch_x)

        # Step 4. Berechne den Fehler mit dem letzten output 
        loss = criterion(output[-1,:,:], batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Berechne den Fehler (Ausgabe des Fehlers alle 100 Iterationen)
    if t % 100 == 0:
        loss_hist.append(loss.item())
        print(t, loss.item())

In [ ]:
plt.figure(1, figsize=(15,10))
plt.plot(loss_hist, label='Loss_Hist', color='b')
plt.legend();

In [ ]:
#reset hidden layer
model.hidden = model.init_hidden(len(dataX))
outputs = model(X.reshape(-1, seq_length, 1).transpose(0,1))
y_pred = torch.max(outputs[-1], 1)[1]

In [ ]:
y_pred

In [ ]:
y

In [ ]:
print('accuracy: ', (y == y_pred).float().mean().item() * 100, '%')